In [ ]:
'''current progress:

I can calculate the Q1+ (B1+) for everything and store it.

todo: Q1-, as well as all future Q2,3,C3,D3,E3,F123, X


write program to expand dataframe so it can be used in machine learning
'''


In [2]:
"""
All necessary imports
"""

import math
import csv

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.spatial.distance as distance
import scipy

from pandas import read_csv
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from minepy import MINE

import matplotlib.pyplot as plt
import seaborn
from mrmr import mrmr_classif
import pprint
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import r2_score

from sklearn.feature_selection import SelectFromModel

from mendeleev import element
from mendeleev.fetch import fetch_table
import pandas as pd
import time
import ast
import re

In [9]:
converters = {
    'elem_in_site_order': ast.literal_eval,
    'elem_by_formula_order': ast.literal_eval,
    'num_electrons': ast.literal_eval,
    'eaffinity': ast.literal_eval,
    'dipole': ast.literal_eval,
    'covalent_rad': ast.literal_eval,
    'atomic_vol': ast.literal_eval,
    'atomic_rad': ast.literal_eval,
    'A1': ast.literal_eval,  # Converter for column 'A1'
    'A2': ast.literal_eval,  # Converter for column 'A2'
    'B1': ast.literal_eval,  # Converter for column 'B1'
    'B2': ast.literal_eval,  # Converter for column 'B2'
    'X': ast.literal_eval,   # Converter for column 'X'
}



df2 = pd.read_csv('Base_descriptors_sorted.csv', converters=converters)
#print(df_read)

In [97]:
pd.set_option('display.max_columns', None)
df = df2
#display(df.head(1))

#print(df['A1'])
print(type(df.head(1)['A1']))
print(df.head(1)['A1'][0]['atomic_rad'])

#remove 21 faulty materials
df = df[df['X'] != {}]
#display(df.iloc[55:66])


<class 'pandas.core.series.Series'>
130.0


In [7]:
'''for each element in elem site order:
    find the index in elem by formula order
    use this index to scrape each atom_rad till num electrons, then add them to a dict in that site

    for me: A1 (aka P1) will be atom radius, atom volume,
            A2 will be dipole polarizability, electron affinity
            A3 will be  covalent radius, num electrons

            
            
in order to read as floats and ints, I will store the method of finding the value as the label of the column

starting out will stick to A-A, A-X, B-X interactions

here are the calculations to see how many columns will be generated
NOTE: IDs from the paper are ABCDEF, since this will be confusing with A and B sites, I will call A and B ids P and Q now:
I am doing 2.5 times the calculations i think
P1:  Atom radius and Atom volume:
P2:  Dipole Polarizability, Electron affinity
P3:  covalent radius, num electrons

the paper is 4,4,6, so I have more features to start with. to prevent this from beign too large an issue I will start by only combining the following:


NOTE: the combinations in the paper are all combinations of two different sites (no A A or B B), with two different symbols to connect them (+ or -).
there are overlaps that are not included (A+B = B+A). this is how they are generated, it took an embarrasing amount of time to verify that

here are the interactions I will consider:

f(A1, A2), f(A2,A1) f(A1,X) f(A2,X) f(X,A1) f(X,A2) f(B1,X) f(B2,X) f(X,B1) f(X,B2)
there are + and - for all of them, which makes the total number for Q1 20, -5 for a1+a2=a2+a1 = 15 This is slightly higher than the 12 from the paper, but manageable

as an example, one of the Q1 descriptors is something like this:
ar = atom radius, av = atom volume

|ar(A1)+ar(A2)|



Q1: absolute differences and sums of P1 : 


this must be very modular, since I am creating order 10,000 columns. 

plan of action:
make list of the 10 interactions that I can loop through (f(A1, A2), f(A2,A1) f(A1,X) f(A2,X) f(X,A1) f(X,A2) f(B1,X) f(B2,X) f(X,B1) f(X,B2))

create a function that takes the two site names as arguments, finds their 2 interactions, and appends them to the respective columns (must compute column name)

calculate(A1, A2, abs_sumdiff, dataframe)

data stored like so:
column name = "P2+", "P2-"

inside column store dict of key value, where key is "ipA1" or "ipA1-ipA2"




'''





<class 'dict'>


In [98]:
dsct = {
    'P1': ['atomic_rad', 'atomic_vol',],
    'P2': ['dipole', 'eaffinity'],
    'P3': ['covalent_rad', 'num_electrons']
}
def get_descripts(id):
    return(dsct[id])

In [123]:
#aA + aB, aA + bB, bA + aB, bA + bB

def add_to_dict(dct, pair, sign, row, descripts):
    if(sign=="+"):
        #print(pair[1])
        str1 = descripts[0] + pair[0] + "+" +descripts[0] + pair[1]# 0 0
        val1 = row[pair[0]][descripts[0]] + row[pair[1]][descripts[0]]
        dct[str1] = val1
        str2 = descripts[0] + pair[0] + "+" +descripts[1] + pair[1]# 0 0
        val2 = row[pair[0]][descripts[0]] + row[pair[1]][descripts[1]]
        dct[str2] = val2
        str3 = descripts[1] + pair[0] + "+" +descripts[0] + pair[1]# 0 0
        val3 = row[pair[0]][descripts[1]] + row[pair[1]][descripts[0]]
        dct[str3] = val3
        str4 = descripts[1] + pair[0] + "+" +descripts[1] + pair[1]# 0 0
        val4 = row[pair[0]][descripts[1]] + row[pair[1]][descripts[1]]
        dct[str4] = val4
    elif(sign=="-"):
        str1 = descripts[0] + pair[0] + "-" +descripts[0] + pair[1]# 0 0
        val1 = abs(row[pair[0]][descripts[0]] - row[pair[1]][descripts[0]])
        dct[str1] = val1
        str2 = descripts[0] + pair[0] + "-" +descripts[1] + pair[1]# 0 0
        val2 = abs(row[pair[0]][descripts[0]] - row[pair[1]][descripts[1]])
        dct[str2] = val2
        str3 = descripts[1] + pair[0] + "-" +descripts[0] + pair[1]# 0 0
        val3 = abs(row[pair[0]][descripts[1]] - row[pair[1]][descripts[0]])
        dct[str3] = val3
        str4 = descripts[1] + pair[0] + "-" +descripts[1] + pair[1]# 0 0
        val4 = abs(row[pair[0]][descripts[1]] - row[pair[1]][descripts[1]])
        dct[str4] = val4
        


#calculate(A1, A2, abs_sumdiff, dataframe)
def calculate(pairlist, id, sign, row):
    descripts = get_descripts(id)
    dct = {}
    for i in pairlist:
        add_to_dict(dct, i, sign, row, descripts)
    return dct
    

In [126]:
#add and sub is both same amount of calculations
cols_to_add = [['Q1+','P1', '+'], ['Q1-','P1', '-'], ['Q2+','P2', '+'],['Q2-','P2', '-'],['Q3+','P3', '+']]#,['Q3-','P3', '-'],] # name, type, sign

pairs = [('A1', 'A2'),('A1', 'X'),('A2', 'X'),('B1', 'X'),('B2', 'X')]

#pairs_sub = [('A1', 'A2'),('A1', 'X'),('A2', 'X'),('B1', 'X'),('B2', 'X'), ('A2', 'A1'), ('X', 'A1'), ('X', 'A2'), ('X', 'B1'), ('X', 'B2')]
for p in cols_to_add:
    df[p[0]] = [{} for _ in range(len(df))]

for(index, row) in df.iterrows():
    for c in cols_to_add:
        if(c[2]=='+'):
            tmp = calculate(pairs, c[1], '+', row)
        elif(c[2]=='-'):
            tmp = calculate(pairs, c[1], '-', row)
        #put tmp in the c[0] column
        df.at[index, c[0]] = tmp

display(df.head(1))
display(df.iloc[5]['Q1-'])

KeyError: 'num_electrons'

KeyError: 'elem_in_site_order'